##INSAID Capstone project - group 1015

###Installing and importing mysql.connector/python

In [92]:
!pip install --user mysql-connector-python

In [0]:
import mysql.connector

###Creating db connection and storing in capdb

In [94]:
 capdbConfig = {
     'host' : 'cpanel.insaid.co',
     'user' : 'student',
      'password' : 'student', 
      'database' : 'Capstone1',
      'connection_timeout' : 180
 }
 capdb = mysql.connector.connect(**capdbConfig)

TypeError: ignored

###Creating python cursor for executing mysql queries, checking tables, structures and count

In [0]:
capdbCursor = capdb.cursor()

In [0]:
capdbCursor.execute("show tables")
capdbTables = capdbCursor.fetchall()
for table in capdbTables:
    print(table)

In [0]:
capdbCursor.execute("describe events_data")
print(capdbCursor.fetchall())

In [0]:
capdbCursor.execute("select count(*) from events_data where state = 'delhi'")
capdbRecords = capdbCursor.fetchall()
for record in capdbRecords:
    print(record)

In [0]:
capdbCursor.execute("describe phone_brand_device_model")
print(capdbCursor.fetchall())

In [0]:
query =   """select count(*)
            from phone_brand_device_model m
            left join events_data e
            on m.device_id = e.device_id
            where e.state = 'delhi'"""
capdbCursor.execute(query, set max_allowed_packet=67108864')
capdbRecords = capdbCursor.fetchall()
for record in capdbRecords:
    print(record)

### Closing db connection

In [0]:
capdb.close()